In [1]:
using LowLevelFEM, LinearAlgebra, SparseArrays
using Tensors

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2), incompatible header (14))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up
[ Info: Precompiling LowLevelFEMTensorsExt [142a104b-3cdc-540a-b57f-68e70732214f] (cache misses: wrong dep version loaded (4))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [2]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube", E=2e2)
prob = Problem([mat], type=:VectorField)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000796136s, CPU 0.000797s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000254217s, CPU 0.00025s)
Info    : Meshing 3D.

Problem("cube1", :VectorField, 3, 3, Material[Material("cube", :Hooke, 200.0, 0.3, 115.38461538461537, 76.92307692307692, 166.66666666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 6859, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [3]:
F = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1]);

In [4]:
#Fleft = nodesToElements(elementsToNodes(F), onPhysicalGroup="left")

#Kext = externalTangentFollowerTL(prob; F=Fleft, traction_phName="left", t_spatial=[1, 0, 0])

In [5]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)


load = BoundaryCondition("right", fy=(x, y, z) -> -(z - 0.5) * 30, fz=(x, y, z) -> (y - 0.5) * 30)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, var"#3#4"(), var"#5#6"(), nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [6]:
μ = mat.μ
λ = mat.λ

115.38461538461537

In [7]:
f_ext0 = loadVector(prob, [load])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [8]:
showDoFResults(f_ext0, name="f")

1

In [9]:
u = vectorField(prob, "cube", [0, 0, 0])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [10]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [11]:
I = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [12]:
P = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])
S = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])

nodal TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [13]:
λ0 = λ
μ0 = μ
C1 = [λ0+2μ0 λ0 λ0 0 0 0;
    λ0 λ0+2μ0 λ0 0 0 0;
    λ0 λ0 λ0+2μ0 0 0 0;
    0 0 0 μ0 0 0;
    0 0 0 0 μ0 0;
    0 0 0 0 0 μ0]

6×6 Matrix{Float64}:
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [15]:
function svk_energy2(C, p)
    λ = p.λ
    μ = p.μ

    I = [1.0 0 0; 0 1 0; 0 0 1]

    E = (C - I) / 2

    return (λ * tr(E)^2 + 2μ * dot(E, E)) / 2
end


svk_energy2 (generic function with 1 method)

In [16]:
using StaticArrays

Cst = SMatrix{3,3}([1.0 0 0; 0 1 0; 0 0 1])
LowLevelFEM._tangent_from_energy(svk_energy2, Cst, (λ=λ, μ=μ))

6×6 SMatrix{6, 6, Float64, 36} with indices SOneTo(6)×SOneTo(6):
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [17]:
C1

6×6 Matrix{Float64}:
 269.231  115.385  115.385   0.0      0.0      0.0
 115.385  269.231  115.385   0.0      0.0      0.0
 115.385  115.385  269.231   0.0      0.0      0.0
   0.0      0.0      0.0    76.9231   0.0      0.0
   0.0      0.0      0.0     0.0     76.9231   0.0
   0.0      0.0      0.0     0.0      0.0     76.9231

In [18]:
using StaticArrays

m = [1.0 0 1; 0 1 0; 0 0 1]
c = m' * m
Cst = SMatrix{3,3}(c)
LowLevelFEM._stress_from_energy(svk_energy2, Cst, (λ=λ, μ=μ))

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 57.6923   0.0      76.9231
  0.0     57.6923    0.0
 76.9231   0.0     134.615

In [19]:
F0 = m
I0 = [1.0 0 0; 0 1 0; 0 0 1]
E0 = (F0' * F0 - I0) / 2
S0 = λ * tr(E0) * I0 + 2μ * E0
#P = F * S

3×3 Matrix{Float64}:
 57.6923   0.0      76.9231
  0.0     57.6923    0.0
 76.9231   0.0     134.615

In [31]:
Kmat = materialTangentMatrix(prob, F=F, energy=svk_energy2, params=(λ=λ, μ=μ))

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.204989736437485, 0.37521645503902656, -1.053710121360494, -0.41794317680739784, -0.20576888274379118, -0.016006181986899598, -2.1677122337624266, 0.5283740736752034, 0.12968663766568872, -0.40366104358059535  …  -24.66122065511127, -6.140804244216471, 60.60440125505186, -191.7848453461899, 2.314193421633088, -0.32808047904574533, 72.39889532605338, 22.03329717628859, -2.287862110930659, 590.0259407213041], 20577, 20577)

In [32]:
C = F' * F
Kgeo = initialStressMatrix(prob, energy=svk_energy2, params=(λ=λ, μ=μ), C=C)

sparse([1, 46, 73, 76, 100, 484, 508, 805, 1375, 1453  …  5832, 5838, 7374, 19599, 19695, 19707, 20511, 20523, 20571, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [0.034551802422288384, 0.008420918264843499, -0.013505314242858626, -0.011194719623159663, -0.006451810810879779, -0.004284325341404555, 0.018196172028112672, -0.0009604438781388281, -0.000981285641560166, 0.011454249467390553  …  0.1272964004558098, -0.19604833125002064, 0.019792940554124058, -0.16240783038341228, -0.1400699376341234, -0.005354260490539763, 0.1373181505470965, -1.2226761942694697, 0.05746022720383115, 3.65662466179301], 20577, 20577)

In [33]:
f_int = internalForceVector(prob, energy=svk_energy2, params=(λ=λ, μ=μ), F=F)

nodal VectorField
[-0.0008951838779163913; -0.0013420966449163554; … ; 0.001360590167344175; -0.0017177088514032288;;]

In [34]:
Kmat0 = materialTangentMatrix(prob, F=F, C=C1)

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [7.204989736437485, 0.37521645503902656, -1.053710121360494, -0.41794317680739784, -0.20576888274379118, -0.016006181986899598, -2.1677122337624266, 0.5283740736752034, 0.12968663766568872, -0.40366104358059535  …  -24.66122065511127, -6.140804244216471, 60.60440125505186, -191.7848453461899, 2.314193421633088, -0.32808047904574533, 72.39889532605338, 22.03329717628859, -2.287862110930659, 590.0259407213041], 20577, 20577)

In [35]:
Kgeo0 = initialStressMatrix(prob, stress=S)

sparse([1, 46, 73, 76, 100, 484, 508, 805, 1375, 1453  …  5832, 5838, 7374, 19599, 19695, 19707, 20511, 20523, 20571, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [0.03455180242228861, 0.008420918264843584, -0.013505314242859586, -0.01119471962315969, -0.006451810810879203, -0.0042843253414048205, 0.018196172028112665, -0.0009604438781388823, -0.0009812856415601034, 0.011454249467390792  …  0.1272964004558098, -0.196048331250022, 0.01979294055412517, -0.16240783038341522, -0.14006993763412462, -0.005354260490536602, 0.1373181505470984, -1.2226761942694646, 0.057460227203828214, 3.6566246617930114], 20577, 20577)

In [36]:
f_int0 = internalForceVector(prob, P=P)

nodal VectorField
[-0.0008674888719643634; -0.0014970987180123736; … ; 0.0018102588192024484; -0.0022397286931662985;;]

In [26]:
norm(Kmat0.A - Kmat.A)

0.0

In [27]:
norm(Kgeo0.A - Kgeo.A)

3.0766424094077427e-12

In [28]:
norm(f_int0.a - f_int.a)

3.230954501580737e-14

In [80]:
#showDoFResults(f_int0, name="f_int_P")
#showDoFResults(f_int, name="f_int_ψ")
#openPostProcessor()

## Innen indul az iteráció

In [29]:
for i in 1:10

    Kmat = materialTangentMatrix(prob, F=F, energy=svk_energy2, params=(λ=λ, μ=μ))

    C = F' * F
    Kgeo = initialStressMatrix(prob, energy=svk_energy2, params=(λ=λ, μ=μ), C=C)

    Kint = Kmat + Kgeo
    Fright = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")

    Kext = externalTangentFollower(prob, [load], F=Fright)
    f_int = internalForceVector(prob, energy=svk_energy2, params=(λ=λ, μ=μ), F=F)
    K = Kint - Kext
    FF = nodesToElements(elementsToNodes(F), onPhysicalGroup="right")
    f_ext = loadVector(prob, [load], F=FF)
    Δu = solveField(
        K,
        f_ext - f_int;
        support=[suppX]
    )

    fixed = constrainedDoFs(prob, [suppX])
    free = freeDoFs(prob, [suppX])
    #norm(Δu.a[:, 1])
    r = f_ext - f_int
    println("‖r_free‖  = ", norm(r.a[free, 1]))
    println("‖r_fixed‖ = ", norm(r.a[fixed, 1]))
    println("‖r‖ = ", norm(r.a[:, 1]))
    println("‖Δu‖ = ", norm(Δu.a[:, 1]))

    u += Δu
    showDoFResults(u)
    F = I + u ∘ ∇
    E = 0.5 * (F' * F - I)
    S = λ * trace(E) * I + 2μ * E
    P = F * S
    #P = λ * trace(F - I) * I + 2μ * (F - I)

    if norm(Δu.a[:, 1]) < 0.01
        break
    end

end

‖r_free‖  = 0.7372768252552513
‖r_fixed‖ = 9.204143066617748e-15
‖r‖ = 0.7372768252552513
‖Δu‖ = 54.215657539331566
‖r_free‖  = 1697.8787188229537
‖r_fixed‖ = 0.5906987134097389
‖r‖ = 1697.8788215761585
‖Δu‖ = 34.30032456590034
‖r_free‖  = 468.5084849438177
‖r_fixed‖ = 0.8117542546371279
‖r‖ = 468.5091881802548
‖Δu‖ = 10.533554121324062
‖r_free‖  = 117.25625812023831
‖r_fixed‖ = 0.5389526695105844
‖r‖ = 117.25749672553958
‖Δu‖ = 21.403047231680425
‖r_free‖  = 25.40070677143489
‖r_fixed‖ = 0.4450954532262452
‖r‖ = 25.40460616602629
‖Δu‖ = 6.905303491942743
‖r_free‖  = 4.028526232079787
‖r_fixed‖ = 0.3851674904008348
‖r‖ = 4.04689728041331
‖Δu‖ = 2.1350955715928617
‖r_free‖  = 1.3496211809132586
‖r_fixed‖ = 0.35511733190514905
‖r‖ = 1.3955592611527217
‖Δu‖ = 3.2845431518339816
‖r_free‖  = 3.7175811152645135
‖r_fixed‖ = 0.3465703967679096
‖r‖ = 3.7337006291998307
‖Δu‖ = 1.8461029982836685
‖r_free‖  = 0.8547046670314885
‖r_fixed‖ = 0.35476358898043864
‖r‖ = 0.92540654412625
‖Δu‖ = 4.635655

In [30]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [28]:
gmsh.finalize()